In [ ]:
# LangChain 1.0+ Runnable Interface
# %pip install langchain>=1.0.0
# %pip install langchain-core>=1.0.0
# %pip install langchain-openai

Now, let's look into the `Runnable` components in more detail. 

`RunnableLambda`

In [ ]:
from langchain.schema.runnable import RunnableLambda


# Create a RunnableLambda and invoke it
runnable = RunnableLambda(lambda x: x*23)
output = runnable.invoke(5)
print(output)  # Output: 115

115


[`RunnableSequence`](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable:~:text=runnablesequence%20invokes%20a%20series%20of%20runnables%20sequentially%2C%20with%20one%20runnable%E2%80%99s%20output%20serving%20as%20the%20next%E2%80%99s%20input.%20construct%20using%20the%20%7C%20operator%20or%20by%20passing%20a%20list%20of%20runnables%20to%20runnablesequence.)

RunnableSequence invokes a series of runnables sequentially, with one runnable’s output serving as the next’s input. Construct using the | operator or by passing a list of runnables to RunnableSequence.

In [ ]:
from langchain_core.runnables import RunnableSequence
# Suppose we have a list of numbers
data = [1, 2, 3, 4, 5]

# Create RunnableLambdas
preprocess_runnable = RunnableLambda(lambda x: [i+1 for i in x])
apply_model_runnable = RunnableLambda(lambda x: x*23)
postprocess_runnable = RunnableLambda(lambda x: sum(x))

# Create a RunnableSequence and invoke it
sequence = preprocess_runnable | apply_model_runnable | postprocess_runnable
# runnable_sequence = RunnableSequence(sequence)
output = sequence.invoke(data)

output

460

In [ ]:
# [1,2,3,4,5]

# # Runnable 1
# [2,3,4,5,6]

# # Runnable 2 
# [2,3,4,5,6] * 23

In [ ]:
print(output)
# or
runnable_sequence = RunnableSequence(first=preprocess_runnable, middle=[apply_model_runnable], last=postprocess_runnable)
runnable_sequence.invoke(data)

460


460

[`Runnableparallel`]([`Runnableparallel`](https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable:~:text=runnableparallel%20invokes%20runnables%20concurrently%2C%20providing%20the%20same%20input%20to%20each.%20construct%20it%20using%20a%20dict%20literal%20within%20a%20sequence%20or%20by%20passing%20a%20dict%20to%20runnableparallel.)https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.base.Runnable.html#langchain_core.runnables.base.Runnable:~:text=runnableparallel%20invokes%20runnables%20concurrently%2C%20providing%20the%20same%20input%20to%20each.%20construct%20it%20using%20a%20dict%20literal%20within%20a%20sequence%20or%20by%20passing%20a%20dict%20to%20runnableparallel.)

RunnableParallel invokes runnables concurrently, providing the same input to each. Construct it using a dict literal within a sequence or by passing a dict to RunnableParallel.

In [ ]:
import langchain
print(langchain.__version__)

0.3.3


In [ ]:
from langchain.schema.runnable import RunnableLambda

# A RunnableSequence constructed using the `|` operator
sequence = RunnableLambda(lambda x: x + 1) | RunnableLambda(lambda x: x * 2)
sequence.invoke(1) # 4
sequence.batch([1, 2, 3]) # [4, 6, 8]

[4, 6, 8]

In [ ]:
# A sequence that contains a RunnableParallel constructed using a dict literal
sequence = RunnableLambda(lambda x: x + 1) | {
    'mul_2': RunnableLambda(lambda x: x * 2),
    'mul_5': RunnableLambda(lambda x: x * 5)
}
sequence.invoke(1) # {'mul_2': 4, 'mul_5': 10}

{'mul_2': 4, 'mul_5': 10}

In [ ]:
from langchain_core.runnables import RunnableParallel

sequence_with_runnable_parallel = RunnableParallel(
    multiply=RunnableLambda(lambda x: x * 2),
    add=RunnableLambda(lambda x: x + 2))

sequence_with_runnable_parallel.invoke(1) # [2, 5]

{'multiply': 2, 'add': 3}

When you build chains, what you're doing is building a type of Runnable!

That could be a RunnableSequence or a RunnableParallel  

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


llm = ChatOpenAI()
prompt = PromptTemplate.from_template("Summarize this {text}")
prompt2 = PromptTemplate.from_template("Explain this {text}")


chain1 = prompt | llm

chain2 = prompt2 | llm

runnable_parallel = RunnableParallel(summary=chain1, explanation=chain2)

runnable_parallel.invoke({"text": "Single responsibility principle in programming"})

{'summary': AIMessage(content='The Single Responsibility Principle states that each class or module in a program should have only one responsibility or reason to change. This helps to keep code clean, maintainable, and easy to understand by separating concerns and preventing classes from becoming too complex.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 16, 'total_tokens': 65, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-3037e21d-9c97-4d96-8352-9cb9068bf6dd-0', usage_metadata={'input_tokens': 16, 'output_tokens': 49, 'total_tokens': 65, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),

In [ ]:
type(runnable_parallel)

langchain_core.runnables.base.RunnableParallel

[RunnableParallel can be useful for manipulating the output of one Runnable to match the input format of the next Runnable in a sequence.](https://python.langchain.com/docs/expression_language/how_to/map#:~:text=RunnableParallel%20can%20be,the%0A%E2%80%9Cquestion%E2%80%9D%20key.)

RunnableParallel (aka. RunnableMap) makes it easy to execute multiple Runnables in parallel, and to return the output of these Runnables as a map.

([see it in docs here](https://python.langchain.com/docs/expression_language/how_to/map#:~:text=runnableparallel%20(aka.%20runnablemap)%20makes%20it%20easy%20to%20execute%20multiple%20runnables%20in%20parallel%2C%20and%20to%20return%20the%20output%20of%20these%20runnables%20as%20a%20map.))

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableParallel

model = ChatOpenAI()
joke_chain = ChatPromptTemplate.from_template("tell me a joke about {topic}") | model
poem_chain = (
    ChatPromptTemplate.from_template("write a 2-line poem about {topic}") | model
)

map_chain = RunnableParallel(joke=joke_chain, poem=poem_chain)

map_chain.invoke({"topic": "bear"})

{'joke': AIMessage(content='Why did the bear break up with his girlfriend? \n\nBecause she was unbearable!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 13, 'total_tokens': 30, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-fa18f369-248b-40d2-9135-4bdddecdc8f5-0', usage_metadata={'input_tokens': 13, 'output_tokens': 17, 'total_tokens': 30, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 'poem': AIMessage(content="In the forest's embrace, the bear roams free\nMajestic and wild, a sight to see.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens':

RunnableParallel are also useful for running independent processes in parallel, since each Runnable in the map is executed in parallel. For example, we can see our earlier joke_chain, poem_chain and map_chain all have about the same runtime, even though map_chain executes both of the other two.

In [ ]:
%%timeit

joke_chain.invoke({"topic": "bear"})

939 ms ± 146 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit

poem_chain.invoke({"topic": "bear"})

860 ms ± 102 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit

joke_chain.invoke({"topic": "bear"})

poem_chain.invoke({"topic": "bear"})

2.19 s ± 528 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit

map_chain.invoke({"topic": "bear"})

1.11 s ± 195 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


[RunnablePassthrough](https://python.langchain.com/docs/expression_language/how_to/passthrough#:~:text=RunnablePassthrough%20allows%20to,pass%20it%20through.)

Allows you to pass inputs unchanged or with addition of new keys.

Usually you would connect this with `RunnableParallel` to assign data to new key in the map.

If you just call it, it will take the input and pass it along.

In [ ]:
from langchain.schema.runnable import RunnablePassthrough

# Create a RunnablePassthrough
passthrough = RunnablePassthrough()
passthrough.invoke(5) # 5

5

But if you call it with the `.assign(...)` mthod, then it will take the input, and add extra arguments passed to the assign function. 

The key being added has to be a lambda function.

In [ ]:
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough

# Passthrough with assignment, adds an extra key-value pair
runnable = RunnablePassthrough.assign(extra_key_add_5=lambda x:  x["num"]+5)
input_data = {"num": 1}
result = runnable.invoke(input_data)
# Output: {'num': 1, 'extra_value': 42}
result

{'num': 1, 'extra_key_add_5': 6}

In combination with RunnableParallel:

In [ ]:
runnable = RunnableParallel(
    origin=RunnablePassthrough(),
    modified=lambda x: x+1
)

runnable.invoke(1) # {'origin': 1, 'modified': 2}

{'origin': 1, 'modified': 2}

In [ ]:
# modified example from langchain docs: https://api.python.langchain.com/en/stable/runnables/langchain_core.runnables.passthrough.RunnablePassthrough.html?highlight=runnablepassthrough#langchain_core.runnables.passthrough.RunnablePassthrough 
# or for an "LLM" example:

def fake_llm(prompt: str) -> str: # Fake LLM for the example
    return "completion"

chain = RunnableLambda(fake_llm) | {
    'original': RunnablePassthrough(), # Original LLM output
    'parsed': lambda text: text[::-1] + " (applied the parsing logic)" # Parsing logic
}

chain.invoke('hello') # {'original': 'completion', 'parsed': 'noitelpmoc'}

{'original': 'completion', 'parsed': 'noitelpmoc (applied the parsing logic)'}

Ok so we have these 4 types of main objects:

- `RunnableLambda`
- `RunnableSequence`
- `RunnablePassthrough`
- `RunnableParallel`

combined they make up kind of the core building block system of langchain allow you to create complex inner logics powered by llms. 

Let's create some fun examples below:

In [ ]:
# LangChain 1.0: Use langchain_openai instead of langchain.chat_models
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

def fun_llm(prompt: str):
    return ChatOpenAI().invoke(f"Make this funny: {prompt}")


def corporate_llm(prompt: str):
    return ChatOpenAI().invoke(f"Make this topic a subject of a very corporate email: {prompt}")

fun_chain = RunnableLambda(fun_llm) | {
    "original-text-input": RunnablePassthrough(), 
    "make-it-corporate": lambda x: corporate_llm(x)
}


fun_chain.invoke("gorillas")

In [ ]:
from langchain.schema.runnable import RunnablePassthrough, RunnableParallel

def fake_llm(prompt: str) -> str: # Fake LLM for the example
    return "completion"

runnable = {
    'llm1':  fake_llm,
    'llm2':  fake_llm,
} | RunnablePassthrough.assign(
    total_chars=lambda inputs: len(inputs['llm1'] + inputs['llm2'])
  )

runnable.invoke('hello')
# {'llm1': 'completion', 'llm2': 'completion', 'total_chars': 20}

{'llm1': 'completion', 'llm2': 'completion', 'total_chars': 20}

Breaking down whats happening:

We create a `runnable` object, which is an instance of the `RunableSequence` object. This chain takes in as input a simple string: `hello`, and what it does is it 
simulates passing that same string to 2 `different` llms (which in this case are the same) and applies the logic of each of these llms as well as a third logic described by the new extra key added by the RunnablePassthrough.assign() method, that takes as input the output of the `llm1` and `llm2` as input to combine it through the lambda function associated with the new extra key created: `total_chars`. 

Let's make this example a bit more interesting by applying some research workflow vibe to it:

In [ ]:
# let's create a couple of llms to do different things with some piece of content.
# In this case let's use different llms to create summarization levels for some text.
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

def llm_summarization_level1(prompt: str):
    return ChatOpenAI().invoke(f"Summarize this text: {prompt}")

def llm_summarization_level2(prompt: str):
    return ChatOpenAI().invoke(f"Summarize this text in bullet points: {prompt}")

def llm_summarization_level3(prompt: str):
    return ChatOpenAI().invoke(f"Summarize this text into one short paragraph: {prompt}")


# now let's create a chain that will run all of these llms in parallel and then return the results

runnable = {
    'level1':  llm_summarization_level1,
    'level2':  llm_summarization_level2,
    'level3':  llm_summarization_level3,
} | RunnablePassthrough.assign(prompt_level4=lambda x: f"Combine these 3 summarizations\
    into one combining all their \n \
    useful features. \
    Level1: {x['level1']}.\n \
    Level2: {x['level2']}.\n\
    Level3: {x['level3']}"
                       ) | RunnablePassthrough.assign(output=RunnableLambda(lambda x: ChatOpenAI().invoke(x['prompt_level4']))) 
# For the assign method with RunningPasshthrough, we'll combine the summarizations into a fourth one.

output = runnable.invoke(
    """
    The code you provided is from the nest_asyncio library, which is designed to patch Python's asyncio library to allow nested usage of the asyncio event loop. To understand this code, it's crucial first to grasp what an event loop is in the context of asynchronous programming.
What is an Event Loop?
An event loop in programming, particularly in asynchronous programming, is a central control structure that manages and dispatches events or messages in a program. In the context of Python's asyncio library, the event loop is a core feature that runs asynchronous tasks and callbacks, handles network IO operations, and manages subprocesses. It is essentially the heart of the asyncio module, enabling asynchronous programming by juggling and scheduling the execution of various tasks.
Key Concepts in the nest_asyncio Code
Patching asyncio: The code modifies (patches) certain parts of the asyncio library. This is done to change the default behavior of asyncio, particularly to support nested event loops, which are not allowed in standard asyncio.
Reentrancy in Event Loops: The primary function of nest_asyncio is to make the asyncio event loop reentrant. In computing, reentrancy refers to the ability of a function to be paused in the middle of execution and safely called again ("re-entered") before its previous executions are complete. This is not normally supported by the asyncio event loops, as they are designed to prevent re-entry (or nesting) to avoid complex problems and unexpected behavior.
Modifying Loop Behavior: The code alters the behavior of the event loop methods like run_forever, run_until_complete, and the internal _run_once. These modifications allow the event loop to pause and resume (re-enter) gracefully, facilitating nesting.
Context Managers: The code uses context managers (manage_run and manage_asyncgens) to properly manage the state of the event loop during entry and exit of asynchronous contexts, which is crucial for handling nested loops correctly.
Task Patching: It also patches the Task class of asyncio to modify its step function. This is necessary to ensure that the tasks (units of work scheduled by the event loop) behave correctly in a nested loop scenario.
Tornado Patching: If the Tornado library (an asynchronous networking library) is used, nest_asyncio makes Tornado aware of the Python asyncio Future, ensuring compatibility.
Understanding the Event Loop in Async Programming
In asynchronous programming, especially in Python's asyncio, the event loop is pivotal. It allows the execution of multiple tasks seemingly in parallel by switching between them. This switching is non-blocking, meaning the program can handle other tasks while waiting for some IO operation to complete, thereby increasing efficiency and responsiveness.

The nest_asyncio library's primary role is to tweak the asyncio's event loop to support nested operation, which is particularly useful in scenarios like running an asyncio event loop inside another asyncio event loop, something that standard asyncio does not support by default.

For more details on asynchronous programming and event loops in Python, the Python asyncio documentation is a comprehensive resource.
"""
)

In [ ]:
from IPython.display import Markdown

Markdown(output['level1'].content)

The text discusses the nest_asyncio library, which patches Python's asyncio library to allow nested event loops. It explains the concept of an event loop in asynchronous programming and how the code modifies the asyncio library to support nested event loops. The code alters the behavior of event loop methods, uses context managers, patches the Task class, and ensures compatibility with the Tornado library. The primary role of nest_asyncio is to enable nested operation of asyncio event loops, increasing efficiency and responsiveness in asynchronous programming.

In [ ]:
Markdown(output['level2'].content)

- The code provided is from the nest_asyncio library, which patches Python's asyncio library for nested event loop usage
- An event loop in programming manages and dispatches events or messages in a program
- Key concepts in the nest_asyncio code include patching asyncio, reentrancy in event loops, modifying loop behavior, using context managers, task patching, and Tornado patching
- The event loop in asynchronous programming allows for the execution of multiple tasks seemingly in parallel
- Nest_asyncio tweaks asyncio's event loop to support nested operation, which standard asyncio does not support by default

In [ ]:
Markdown(output['level3'].content)

The text discusses the role of the nest_asyncio library in patching Python's asyncio library to enable nested usage of the event loop. It explains the concept of event loops in asynchronous programming and how the nest_asyncio code modifies asyncio to support nested event loops. By making the event loop reentrant and modifying loop behavior, the code allows for graceful nesting and proper management of asynchronous contexts. Additionally, it patches the Task class and ensures compatibility with the Tornado library. Overall, understanding the event loop is crucial in asynchronous programming for efficient and responsive task execution.

A simple research assistant, heavily based on this implementation by Harrison Chase:
- https://gist.github.com/hwchase17/69a8cdef9b01760c244324339ab64f0c

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
import requests
from bs4 import BeautifulSoup
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
import os
from langchain.tools import tool
from serpapi import GoogleSearch
import json

RESULTS_PER_QUESTION = 1

serpapi_params = {
    "engine": "google",
    "api_key": os.environ["SERPAPI_KEY"]
}

def web_search(query: str) -> str:
    """Finds general knowledge information using Google search. Can also be used
    to augment more 'general' knowledge to a previous specialist query."""
    search = GoogleSearch({**serpapi_params, "q":query, "n": 3})
    results = search.get_dict()["organic_results"]
    urls = [r["link"] for r in results]
    
    return urls


def scrape_text(url: str):
    # Send a GET request to the webpage
    try:
        response = requests.get(url)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the content of the request with BeautifulSoup
            soup = BeautifulSoup(response.text, "html.parser")

            # Extract all text from the webpage
            page_text = soup.get_text(separator=" ", strip=True)

            # Print the extracted text
            return page_text
        else:
            return f"Failed to retrieve the webpage: Status code {response.status_code}"
    except Exception as e:
        print(e)
        return f"Failed to retrieve the webpage: {e}"


def collapse_list_of_lists(list_of_lists):
    content = []
    for l in list_of_lists:
        content.append("\n\n".join(l))
    return "\n\n".join(content)

In [ ]:
SUMMARY_TEMPLATE = """{text} 
-----------
Using the above text, answer in short the following question: 
> {question}
-----------
if the question cannot be answered using the text, imply summarize the text. Include all factual information, numbers, stats etc if available."""  # noqa: E501
SUMMARY_PROMPT = ChatPromptTemplate.from_template(SUMMARY_TEMPLATE)


url = "https://python.langchain.com/docs/concepts/"

scrape_and_summarize_chain = RunnablePassthrough.assign(
    summary = RunnablePassthrough.assign(
    text=lambda x: scrape_text(x["url"])[:10000]
) | SUMMARY_PROMPT | ChatOpenAI(model="gpt-4o-mini") | StrOutputParser()
) | (lambda x: f"URL: {x['url']}\n\nSUMMARY: {x['summary']}")

At this stage, even though this chain requires two keys in the input dictionary, that's not reflected in the input_schema() of the chain, because in it only goes the variables contained in the ChatPromptTemplate(). 

Interestingly enough, the `text` variable which is in the ChatPromptTemplate, is not in the schema because its already part of the chain itself (therefore not being necessary in the input_schema() I guess).

In [ ]:
# At this
# let's test this chain on some url
scrape_and_summarize_chain.invoke({"question": "What are some important concepts in langchain?", "url": url})

'URL: https://python.langchain.com/docs/concepts/\n\nSUMMARY: Some important concepts in LangChain include:\n\n1. **Chat Models**: LLMs that process sequences of messages and output messages.\n2. **Messages**: Units of communication in chat models representing input and output.\n3. **Chat History**: A conversation represented as a sequence of alternating user messages and model responses.\n4. **Tools**: Functions with schemas defining their name, description, and arguments.\n5. **Tool Calling**: A chat model API that accepts tool schemas and messages, returning invocations of those tools.\n6. **Structured Output**: A technique for models to respond in structured formats like JSON.\n7. **Memory**: Persistence of conversation information for future use.\n8. **Multimodality**: Working with various data forms like text, audio, images, and video.\n9. **Runnable Interface**: The base abstraction for many LangChain components.\n10. **Streaming**: APIs that surface results as they are generate

In [ ]:
web_search_chain = RunnablePassthrough.assign(
    urls = lambda x: web_search(x["question"]) # urls will be the output of the web_search function()
) | (lambda x: [{"question": x["question"], "url": u} for u in x["urls"]]) | scrape_and_summarize_chain.map()

Let's now test this web search chain.

In [ ]:
web_search_chain.invoke({"question": "Look up tool calling in langchain"})

["URL: https://python.langchain.com/docs/concepts/tool_calling/\n\nSUMMARY: Tool calling in LangChain allows AI models to interact with external systems, such as databases or APIs, by using a structured input schema. It involves four key concepts:\n\n1. **Tool Creation**: Tools are created using the `@tool` decorator, which associates a function with its input schema.\n2. **Tool Binding**: The created tool must be connected to a model that supports tool calling, allowing the model to understand the tool and its required input.\n3. **Tool Calling**: The model can decide when to utilize a tool, ensuring its response matches the tool's input schema.\n4. **Tool Execution**: The tool is executed with the arguments provided by the model.\n\nA typical workflow includes creating tools, binding them to a model, and invoking the model with user input, which may lead to tool calls. An example tool is a function that multiplies two integers.\n\nFor more details, refer to the model integrations tha

In [ ]:
SEARCH_PROMPT = ChatPromptTemplate.from_messages(
    [
        (
            "user",
            "Write 3 google search queries to search online that form an "
            "objective opinion from the following: {question}\n"
            "You must respond with a list of strings in the following format: "
            '["query 1", "query 2", "query 3"].',
        ),
    ]
)

search_question_chain = SEARCH_PROMPT | ChatOpenAI(temperature=0) | StrOutputParser() | json.loads

In [ ]:
search_question_chain.invoke({"question": "What are the most useful langchain utilities for research?"})

['best langchain utilities for research',
 'top langchain tools for academic research',
 'recommended langchain resources for scholarly work']

In [ ]:
full_research_chain = search_question_chain | (lambda x: [{"question": q} for q in x]) | web_search_chain.map()

In [ ]:
full_research_chain.invoke({"question": "What is langchain?"})

[['URL: https://www.langchain.com/\n\nSUMMARY: LangChain is a composable framework designed to build applications powered by large language models (LLMs). It supports developers throughout the LLM application lifecycle, allowing them to create context-aware and reasoning applications that leverage company data and APIs. LangChain products include LangGraph, which orchestrates agent-driven workflows, and LangSmith, an enterprise platform for debugging, testing, deploying, and monitoring LLM applications. LangChain products have seen over 20 million monthly downloads, with more than 100,000 apps powered and 100,000 GitHub stars. The platform is used by a large developer community to enhance operational efficiency, increase personalization, and deliver revenue-generating products.',
  'URL: https://aws.amazon.com/what-is/langchain/\n\nSUMMARY: LangChain is an open-source framework designed for building applications based on large language models (LLMs). It provides tools and abstractions 

In [ ]:
WRITER_SYSTEM_PROMPT = "You are an AI critical thinker research assistant. Your sole purpose is to write well written, critically acclaimed, objective and structured reports on given text."  # noqa: E501


# Report prompts from https://github.com/assafelovic/gpt-researcher/blob/master/gpt_researcher/master/prompts.py
RESEARCH_REPORT_TEMPLATE = """Information:
--------
{research_summary}
--------
Using the above information, answer the following question or topic: "{question}" in a detailed report -- \
The report should focus on the answer to the question, should be well structured, informative, \
in depth, with facts and numbers if available and a minimum of 1,200 words.
You should strive to write the report as long as you can using all relevant and necessary information provided.
You must write the report with markdown syntax.
You MUST determine your own concrete and valid opinion based on the given information. Do NOT deter to general and meaningless conclusions.
Write all used source urls at the end of the report, and make sure to not add duplicated sources, but only one reference for each.
You must write the report in apa format.
Please do your best, this is very important to my career."""  # noqa: E501

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", WRITER_SYSTEM_PROMPT),
        ("user", RESEARCH_REPORT_TEMPLATE),
    ]
)


chain = RunnablePassthrough.assign(
    research_summary= full_research_chain | collapse_list_of_lists
) | prompt | ChatOpenAI(model="gpt-4o-mini") | StrOutputParser()


`RunnableMap`

> Let's modify this example for something that uses the updated version of the pydantic library.

In [ ]:
# this example requires pydantic==
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableMap
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser


llm_chat = ChatOpenAI()

vectorstore = Chroma.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

retriever.get_relevant_documents("Where did harrison work?")

template = "Answer the question based on this context: {context}\n\nQuestion: {question}"
prompt = ChatPromptTemplate.from_template(template)

output_parser = StrOutputParser()

chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"],
    
}) | prompt | llm_chat | output_parser



chain.invoke({"question": "What did harrison do?"})

/var/folders/5l/y8s3fc655417629rqwgxkhx80000gn/T/ipykernel_31597/2928158330.py:18: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("Where did harrison work?")
Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2
Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


'Harrison worked at Kensho.'

If we inspect just the `RunnableMap`:

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
inputs_with_lambda = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"],
})

inputs_with_lambda.invoke({"question": "What did harrison do?"})

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


{'context': [Document(metadata={}, page_content='harrison worked at kensho'),
  Document(metadata={}, page_content='bears like to eat honey')],
 'question': 'What did harrison do?'}

`RunnableMap` is an alias for `RunnableParallel`.

In [ ]:
inputs_with_runnable_passthrough = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"],
})

inputs_with_runnable_passthrough.invoke({"question": "What did harrison do?"})

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


{'context': [Document(metadata={}, page_content='harrison worked at kensho'),
  Document(metadata={}, page_content='bears like to eat honey')],
 'question': 'What did harrison do?'}

You get the same thing!

You can also use tools with runnables in some neat and interesting ways. Let's see that in the next notebook (2.2)

We see that what the `RunnableMap` is doing, is create the right inputs for the downstream of this chain which in this case is seeting the `context` key with a value of a list of `Document` objects, and the question key with itself.
(which now we can probably change to something simpler with the `RunnablePassthrough` for example).